In [10]:
import cv2
import numpy as np
from keras.datasets import mnist
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
import matplotlib.pyplot as plt

In [11]:
# Load the dateset from MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshaping the dataset to fit our model
X_train = X_train.reshape(60000, 28, 28, 1)
X_test = X_test.reshape(10000, 28, 28, 1)

# y_train and y_test use one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

## Declare the model
model = Sequential()

## Declare the layers
layer_1 = Conv2D(64, kernel_size=3, activation='relu', input_shape=(28, 28, 1))
layer_2 = MaxPooling2D(pool_size=2)
layer_3 = Conv2D(32, kernel_size=3, activation='relu')
layer_4 = MaxPooling2D(pool_size=2)
layer_5 = Dropout(0.5)
layer_6 = Flatten()
layer_7 = Dense(128, activation="relu")
layer_8 = Dropout(0.5)
layer_9 = Dense(10, activation='softmax')

# Add the layers to the model
model.add(layer_1)
model.add(layer_2)
model.add(layer_3)
model.add(layer_4)
model.add(layer_5)
model.add(layer_6)
model.add(layer_7)
model.add(layer_8)
model.add(layer_9)

# Compile and run the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

example = X_train[1]
prediction = model.predict(example.reshape(1, 28, 28, 1))

# Output prediction using the softmax functioin
print ("Prediction (Softmax) from the neural network:\n\n {}".format(prediction))
# Output prediction using the hardmax functioin
hard_maxed_prediction = np.zeros(prediction.shape)
hard_maxed_prediction[0][np.argmax(prediction)] = 1
print ("\n\nHard-maxed form of the prediction: \n\n {}".format(hard_maxed_prediction))

# Final output (the highest probability)
print("\n\nFinal Output: {}".format(np.argmax(prediction)))

Epoch 1/3
1875/1875 [==============================] - 12s 6ms/step - loss: 0.9239 - accuracy: 0.7643 - val_loss: 0.1435 - val_accuracy: 0.9608
Epoch 2/3
1875/1875 [==============================] - 13s 7ms/step - loss: 0.3106 - accuracy: 0.9075 - val_loss: 0.0908 - val_accuracy: 0.9734
Epoch 3/3
1/1 [==============================] - 0s 93ms/step
Prediction (Softmax) from the neural network:

 [[9.9994850e-01 4.6220253e-10 1.9955785e-06 4.0811430e-09 3.8568332e-10
  1.9359177e-08 2.1382518e-06 3.3735027e-08 4.7196754e-05 9.4854897e-08]]


Hard-maxed form of the prediction: 

 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


Final Output: 0


In [12]:
digits = []

# Read the image and convert to greyscale
image = cv2.imread('./test1.png')
grey = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)

# Thresholding the image
ret, thresh = cv2.threshold(grey.copy(), 200, 255, cv2.THRESH_BINARY_INV)

# Extract the contours
contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

preprocessed_digits = []

for c in contours:

    x,y,w,h = cv2.boundingRect(c)
    # Creating a rectangle around the digit in the original image (for displaying the digits fetched via contours)
    cv2.rectangle(image, (x,y), (x+w, y+h), color=(0, 255, 0), thickness=2)
    # Cropping out the digit from the image corresponding to the current contours in the for loop
    digit = thresh[y:y+h, x:x+w]
    # Resizing that digit to (18, 18)
    resized_digit = cv2.resize(digit, (18,18))
    # Padding the digit with 5 pixels of black color (zeros) in each side to finally produce the image of (28, 28)
    padded_digit = np.pad(resized_digit, ((5,5),(5,5)), "constant", constant_values=0)
    # Adding the preprocessed digit to the list of preprocessed digits
    preprocessed_digits.append(padded_digit)

    
inp = np.array(preprocessed_digits)
for digit in preprocessed_digits:
    # Use prediction model
    prediction = model.predict(digit.reshape(1, 28, 28, 1))  
    print("\n\nFinal Output: {}".format(np.argmax(prediction)))
    digits.append(np.argmax(prediction))

1/1 [==============================] - 0s 35ms/step


Final Output: 7
1/1 [==============================] - 0s 39ms/step


Final Output: 3
1/1 [==============================] - 0s 28ms/step


Final Output: 7
1/1 [==============================] - 0s 30ms/step


Final Output: 0
1/1 [==============================] - 0s 27ms/step


Final Output: 0
1/1 [==============================] - 0s 41ms/step


Final Output: 5
1/1 [==============================] - 0s 29ms/step


Final Output: 3
1/1 [==============================] - 0s 31ms/step


Final Output: 2
1/1 [==============================] - 0s 30ms/step


Final Output: 9
1/1 [==============================] - 0s 30ms/step


Final Output: 4


In [13]:
print(digits)

[7, 3, 7, 0, 0, 5, 3, 2, 9, 4]
